# Tweet Sentiment Analyzer

Made by: Rehan Adi Satrya - 13518061

Tujuan dari aplikasi ini adalah agar pengguna dapat mengetahui apakah ketikan mereka pada media sosial berpotensi menyakiti seseorang

In [197]:
#Import Library yang dibutuhkan
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random
import tensorflow as tf

from numpy import array
from sklearn.model_selection import train_test_split

from tensorflow.keras import regularizers
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences


In [198]:
# Load Dataset
data = pd.read_csv('dataset_two.csv', index_col=0)
data = data[['Instagram Comment Text', 'Sentiment']]
data.head()

,Instagram Comment Text,Sentiment
Id,,
1,<USERNAME> TOLOL!! Gak ada hubungan nya kegug...,negative
2,Geblek lo tata...cowo bgt dibela2in balikan......,negative
3,Kmrn termewek2 skr lengket lg duhhh kok labil ...,negative
4,"Intinya kalau kesel dengan ATT nya, gausah ke ...",negative
5,"hadewwwww permpuan itu lg!!!!sakit jiwa,knp ha...",negative


In [199]:
y = [0 if i=='negative' else 1 for i in data['Sentiment']]
x = data['Instagram Comment Text']

In [200]:
# Split Data

X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.30)

In [201]:
(unique, counts) = np.unique(y_test, return_counts=True)
frequencies = np.asarray((unique, counts)).T
print(frequencies)

[[ 0 55]
 [ 1 65]]


In [202]:
# Model Configuration

vocab_size = 10000
embedding_dim = 16
max_length = 100
trunc_type='post'
padding_type='post'
oov_tok = "<OOV>"

In [203]:
# Preprocessing - Tokenization

tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(x)

word_index = tokenizer.word_index

training_sequences = tokenizer.texts_to_sequences(X_train)
testing_sequences = tokenizer.texts_to_sequences(X_test)

# Padding
training_padded = pad_sequences(training_sequences, maxlen=max_length, padding='post', truncating='post')
testing_padded = pad_sequences(testing_sequences, maxlen=max_length, padding='post', truncating='post')

# COnvert to Numpy Array
training_padded = np.array(training_padded)
training_labels = np.array(y_train)

testing_padded = np.array(testing_padded)
testing_labels = np.array(y_test)

In [204]:
# Model
model = tf.keras.Sequential([
                             tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
                             tf.keras.layers.GlobalAveragePooling1D(),
                             tf.keras.layers.Dense(24, activation='sigmoid'),
                             tf.keras.layers.Dropout(0.5),
                             tf.keras.layers.Dense(1, activation='sigmoid')
])
model.compile(loss='binary_crossentropy', optimizer='nadam', metrics=['accuracy'])

In [205]:
# Training

num_epoch = 100
history = model.fit(training_padded, training_labels, batch_size = 32, epochs=num_epoch)
print(history.history.keys())

Epoch 1/100
9/9 [==============================] - 1s 5ms/step - loss: 0.7595 - accuracy: 0.4821
Epoch 2/100
9/9 [==============================] - 0s 4ms/step - loss: 0.7327 - accuracy: 0.5250
Epoch 3/100
9/9 [==============================] - 0s 4ms/step - loss: 0.7604 - accuracy: 0.4964
Epoch 4/100
9/9 [==============================] - 0s 5ms/step - loss: 0.7265 - accuracy: 0.5321
Epoch 5/100
9/9 [==============================] - 0s 3ms/step - loss: 0.7382 - accuracy: 0.4714
Epoch 6/100
9/9 [==============================] - 0s 4ms/step - loss: 0.7404 - accuracy: 0.5107
Epoch 7/100
9/9 [==============================] - 0s 4ms/step - loss: 0.7561 - accuracy: 0.4750
Epoch 8/100
9/9 [==============================] - 0s 4ms/step - loss: 0.7346 - accuracy: 0.5321
Epoch 9/100
9/9 [==============================] - 0s 4ms/step - loss: 0.7585 - accuracy: 0.4893
Epoch 10/100
9/9 [==============================] - 0s 4ms/step - loss: 0.7416 - accuracy: 0.4964
Epoch 11/100
9/9 [===========

In [207]:
import re
def preprocessing(string):
  # Jadikan lowercase semua
  string = string.lower()
  # mention
  string = re.sub("@[A-Za-z0-9]+", '', string)
  # Hapus link dari tweet
  string = re.sub(r'https?://\S+', '', string)
  # Hapus non letter seperti emoji
  string = re.sub(r"[^a-z\s\(\-:\)\\\/\];='#]", '', string)
  # Hashtag
  string = re.sub('#|_', '', string)
  return string

dor = preprocessing("hahahaha @redi aneh bangettt #hehe https://www.google.com")
print(dor)

hahahaha  aneh bangettt hehe 


In [217]:
tweet = """ ini orang omongannya gak bisa dijaga kali ya @akunbuzzer """
sentence = [preprocessing(tweet)]
sequences = tokenizer.texts_to_sequences(sentence)
padded = pad_sequences(sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)
res = np.rint(model.predict(padded))
if(res == [[0.]]):
  print("Negatif")
else:
  print("Positif")

Negatif
